In [22]:
import numpy as np
from tqdm.notebook import tqdm
from csv import writer

np.set_printoptions(suppress=True)

import tensorflow as tf
from tensorflow import keras
tf.config.run_functions_eagerly(True)

In [23]:
@tf.function
def update_step(batch_images, batch_labels, model, optimizer, loss_fn):
    with tf.GradientTape() as tape:

        ### Forward pass
        label_pred = model(batch_images)

        ### Compute loss
        loss = loss_fn(y_true=batch_labels, y_pred=label_pred)

    ### Compute gradients w.r.t batch loss
    grads = tape.gradient(loss, model.trainable_variables)

    ### Update trainable parameters
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [24]:
def train_epoch_noisy(dataset, model, optimizer, loss_fn, pbar, noise_level, noise_amount):
    losses = []
    for (batch_img, batch_labels) in dataset:
        # Check if a random number is less than noise_amount
        # This will result in noise_amount of the images being given noise
        if (np.random.rand() < noise_amount): 
            # if the image is to be given noise, noise_level decides the strength of the noise
            image = batch_img + noise_level * np.random.normal(size=batch_img.shape)
        else:
            # if the image is not given noise, simply take the regular image
            image = batch_img
        # compute the loss and update as usual
        loss = update_step(image, batch_labels, model, optimizer, loss_fn)
        losses.append(loss.numpy())
        pbar.set_postfix_str(f'Loss: {loss.numpy():.3f}', refresh=False)
        pbar.update(1)
    return losses

In [25]:
def test_robustness(model, noise_levels, test_images, test_labels):
    accuracies = []
    for noise_level in noise_levels:
        corrupted_images = test_images + noise_level * np.random.normal(size=test_images.shape)
        preds = model.predict(corrupted_images)
        accuracy = (preds.argmax(1) == test_labels).sum() / test_labels.shape[0]
        accuracies.append(accuracy)
    return accuracies

In [26]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

train_images = train_images / 255
test_images = test_images / 255
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [27]:
# Create a TF data set
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(buffer_size=train_images.shape[0])
dataset = dataset.batch(32)

2025-04-21 16:52:42.162674: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 376320000 exceeds 10% of free system memory.


In [38]:
min_noise_level = 0
max_noise_level = 0.5
num_noise_levels = 50
test_noise_levels = np.linspace(min_noise_level, max_noise_level, num_noise_levels)

# Parameters for training
train_noise_amount = 0.5
train_noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
epochs = 5

In [39]:
# This was meant to be in a loop through the different noise levels and amounts but my laptop didn't like that
for train_noise_level in train_noise_levels:
    # Create loss object
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    # Create optimizer
    optimizer = tf.keras.optimizers.Adam(1e-4)

    # Create a new model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Train the model
    for ep in range(1, epochs+1):
        with tqdm(total=len(dataset), desc=f"Training epoch {ep}", mininterval=0.1) as pbar:
            l = train_epoch_noisy(dataset, model, optimizer, loss_fn, pbar, train_noise_level, train_noise_amount)

    # Test the model
    accuracies = test_robustness(model, test_noise_levels, test_images, test_labels)

    # Save accuracies to a CSV
    # Open file in append mode
    with open("robustness.csv", 'a') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(accuracies)

Training epoch 1:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 2:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 3:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 4:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 5:   0%|          | 0/1875 [00:00<?, ?it/s]

...

Training epoch 2:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 3:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 4:   0%|          | 0/1875 [00:00<?, ?it/s]

Training epoch 5:   0%|          | 0/1875 [00:00<?, ?it/s]

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15